## DPENet

#### Import module

In [1]:
import glob
import json
import os
import numpy as np
import math
from tqdm.notebook import trange, tqdm
from time import sleep
import torch
import torch.nn as nn
from torch import optim
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
from tensorboardX import SummaryWriter

#### Load Input and GT

GT format  
- ['ego_global_x', 'ego_global_y']  

Input format  
- ['ego_vx', 'yaw_rate', 'sas_angle', 'long_acc', 'lat_acc']

In [2]:
gt_folder = './DDPE_GT_adaf_1/'
in_folder = './DDPE_input_adaf_1/'
gt_file_list = glob.glob(gt_folder+'/*_gt.json')
gt_file_list.sort()
in_train, gt_train = [], []
in_test, gt_test = [], []

In [ ]:
# Load the json log.
for k in trange(len(gt_file_list),desc='folder progress'):
    # Load the GT
    file_name_gt = gt_file_list[k]
    with open(file_name_gt, 'r', encoding='utf-8-sig') as data_file_gt:
        try:
            data_gt = json.load(data_file_gt)
        except:
            continue
    
    # Load the input
    file_name_in = in_folder + gt_file_list[k][17:26] + '_in.json'
    with open(file_name_in, 'r', encoding='utf-8-sig') as data_file_in:
        try:
            data_in = json.load(data_file_in)
        except:
            continue
    if k % 10 == 0:
        gt_test.append([data_gt['ego_global_x'], data_gt['ego_global_y']])
        in_test.append([data_in['ego_vx'],data_in['yaw_rate'],data_in['sas_angle'],data_in['long_acc'],data_in['lat_acc']])
    else:
        gt_train.append([data_gt['ego_global_x'], data_gt['ego_global_y']])
        in_train.append([data_in['ego_vx'],data_in['yaw_rate'],data_in['sas_angle'],data_in['long_acc'],data_in['lat_acc']])

folder progress:   0%|          | 0/533745 [00:00<?, ?it/s]

In [ ]:
# make list to tensor
gt_train = torch.tensor(gt_train).cuda()
in_train = torch.tensor(in_train).cuda()
gt_test = torch.tensor(gt_test).cuda()
in_test = torch.tensor(in_test).cuda()

In [ ]:
# class to use the dataloader
class DPEDataset(data.Dataset):
    def __len__(self):
        return len(in_train)
    
    def __getitem__(self,idx):
        in_tmp = in_train[idx]
        gt_tmp = gt_train[idx]
        return {'in':in_tmp,'gt':gt_tmp}

In [ ]:
class DPETestset(data.Dataset):
    def __len__(self):
        return len(in_test)
    
    def __getitem__(self,idx):
        in_tm = in_test[idx]
        gt_tm = gt_test[idx]
        return {'in':in_tm,'gt':gt_tm}

In [ ]:
dpe_dataset = DPEDataset()
dpe_testset = DPETestset()

In [ ]:
len(gt_train), len(in_train), len(gt_test), len(in_test)

In [ ]:
# Exercise

#### Loss function

In [ ]:
# get lateral point from the polynomial equation
# input 
# : longitudinal distance, C1, C2, C3
# output 
# : lateral point x

def get_x_polynomial(long_dist, C1, C2, C3):
    mean = 0.0
    inv_sigma = 1.0
    y_ = (long_dist-mean) * inv_sigma
    
    x = C1 * (y_**1) + C2 * (y_**2) + C3 * (y_**3)
    
    return x

In [ ]:
# MSELoss from lateral point
# input
# : [C1, C2, C3], gt['ego_global_x', 'ego_global_y']
# output
# : sum, variance, standard deviation of MSELoss

def lp_loss(output, gt):
    criterion = nn.MSELoss()
    # criterion = nn.CrossEntropyLoss()
    mse_total = []
    mse_mean, mse_var, mse_std = 0.0, 0.0, 0.0
    for i in range(len(gt[0][1])):
        x_tmp = get_x_polynomial(gt[0][1][i],output[0][0],output[0][1],output[0][2])
        # make to tensor if needed
        x_tmp_torch = x_tmp
        gt_tmp_torch = gt[0][0][i]
        
        mse_tmp = criterion(x_tmp_torch,gt_tmp_torch)
        mse_total.append(mse_tmp)
    mse_mean = torch.mean(torch.tensor(mse_total))
    mse_var = torch.var(torch.tensor(mse_total))
    mse_std = torch.std(torch.tensor(mse_total))
    
    return mse_mean, mse_var, mse_std

In [ ]:
# Exercise

#### DPENet

In [ ]:
# DPENet
# input
# : CAN information (ego_vx, yaw_rate, sas_angle, long_acc, lat_acc)
# output
# : variable for polynomial equation (C1, C2, C3)

# construct model on cuda if available
use_cuda = torch.cuda.is_available()

class DPENet(nn.Module):
    def __init__(self):
        super(DPENet,self).__init__()
        # fullyconncted layer
        fc1 = nn.Linear(5,10,bias=True)
        fc2 = nn.Linear(10,10,bias=True)
        fc3 = nn.Linear(10,10,bias=True)
        fc4 = nn.Linear(10,10,bias=True)
        fc5 = nn.Linear(10,10,bias=True)
        fc6 = nn.Linear(10,10,bias=True)
        fc7 = nn.Linear(10,10,bias=True)
        fc8 = nn.Linear(10,10,bias=True)
        fc9 = nn.Linear(10,10,bias=True)
        fc10 = nn.Linear(10,3,bias=True)
        tanh = nn.Tanh()
        dropout = nn.Dropout(0.3)
        #self.relu = nn.ReLU()
        
        self.fc_module = nn.Sequential(
            fc1,
            tanh,
            fc2,
            tanh,
            fc3,
            tanh,
            fc4,
            tanh,
            fc5,
            tanh,
            fc6,
            tanh,
            fc7,
            tanh,
            fc8,
            tanh,
            fc9,
            tanh,
            fc10
        )
        # use gpu
        if use_cuda:
            self_fc_module = self.fc_module.cuda()
        
    def forward(self,x):
        out = self.fc_module(x)
        
        return out

In [ ]:
print(DPENet())

In [ ]:
# Exercise

#### Train

In [ ]:
# Xavier Normal initialization
def weight_init(m): 
    if isinstance(m, nn.Linear):
        size = m.weight.size()
        fan_out = size[0] # number of rows
        fan_in = size[1] # number of columns
        variance = np.sqrt(2.0/(fan_in + fan_out))
        m.weight.data.normal_(0.0, variance)

In [ ]:
# argument
# learning rate and beta1,2 for optimizer
dpenet_learning_rate = 0.01 # 0.01(x) 0.001 0.0002
dpenet_momentum = 0.9
dpenet_beta1 = 0.5
dpenet_beta2 = 0.999
# lambda for loss
lambda_mean = 10**-10
lambda_var = 10**-10
lambda_std = 10**-10
# number of epoch and log steo
num_epochs = 1000
log_step = 1000

In [ ]:
# dataloader
data_loader = data.DataLoader(dataset=dpe_dataset, batch_size=1, shuffle=True)
test_loader = data.DataLoader(dataset=dpe_testset, batch_size=1)

# tensorboardX summary
summary = SummaryWriter('log')
dpenet = DPENet()
dpenet.apply(weight_init)

# To make graph in summary
dummmy = torch.zeros(5)
summary.add_graph(dpenet,dummmy.cuda())

# Optimizer
dpenet_optimizer = optim.Adam(dpenet.parameters(), lr=dpenet_learning_rate)
#dpenet_optimizer = optim.Adam(dpenet.parameters(), dpenet_learning_rate, [dpenet_beta1, dpenet_beta2])
#dpenet_optimizer = optim.SGD(dpenet.parameters(), lr=dpenet_learning_rate, momentum=dpenet_momentum)

for epoch in range(num_epochs):
    for i, sample in enumerate(data_loader):
        
        # input : ['ego_vx', 'yaw_rate', 'sas_angle', 'long_acc', 'lat_acc']
        # gt : ['ego_global_x', 'ego_global_y']
        input_ = sample['in'].cuda()
        gt_ = sample['gt']
        output_ = dpenet(input_)
        #print(output_)
        # train dpenet
        # reset the gradient for the optimizer
        # zero_grad : 역전파 실행 전 변화도 0으로
        dpenet_optimizer.zero_grad()
        
        # loss from lateral position
        loss_lp_mean, loss_lp_var, loss_lp_std = lp_loss(output_,gt_)
        loss = lambda_mean*loss_lp_mean + lambda_var*loss_lp_var + lambda_std*loss_lp_std
        loss = Variable(loss, requires_grad=True)
        # backpropagate the gradient
        loss.backward()
        # update the weights using the gradient with the optimizer
        dpenet_optimizer.step()
        
        # print the log information
        if (i+1) % log_step == 0:
            print('Epoch [%d/%d], BatchStep[%d/%d]' % (epoch + 1, num_epochs, i + 1, len(in_train)))
            print('Loss: %.4f, lp_mean: %.4f, lp_var: %.4f, lp_std: %.4f' 
                  % (loss.data, loss_lp_mean.data, loss_lp_var.data, loss_lp_std.data))

    loss_test = 0
    loss_test_lsit = []
    with torch.no_grad():
        for i, sample_t in enumerate(test_loader):
            input_t = sample_t['in'].cuda()
            gt_t = sample_t['gt']
            output_t = dpenet(input_t)
            loss_lp_mean_t, loss_lp_var_t, loss_lp_std_t = lp_loss(output_t,gt_t)
            loss_test_lsit.append(loss_lp_mean_t)
            
    loss_test = torch.mean(torch.tensor(loss_test_lsit))
    print('Epoch [%d/%d]' % (epoch + 1, num_epochs))
    print('Test Loss: %.4f' % (loss_test.data))

    # visualize the parameter
    for name, param in dpenet.named_parameters():
        #summary.add_histogram(name, param, epoch)
        summary.add_histogram(name, param.clone().cpu().data.numpy(), epoch, bins='doane')
    
    # save summary
    summary.add_scalar('loss/loss', loss.item(), epoch)
    summary.add_scalar('loss/loss_lp_mean', loss_lp_mean.item(), epoch)
    summary.add_scalar('loss/loss_lp_var', loss_lp_var.item(), epoch)
    summary.add_scalar('loss/loss_lp_std', loss_lp_std.item(), epoch)
    summary.add_scalar('loss_t/loss', loss_test.item(), epoch)
    
    summary.flush()
    # save the model parameters
    #if epoch % 10 == 0:
    model_path = os.path.join('./DPENet_model', 'DPENet-%d.pkl' % (epoch + 1))
    torch.save(dpenet.state_dict(), model_path)

In [ ]:
# Exercise

In [ ]:
# with 
epoch [1/1000] Test Loss: 236042076160.0000
epoch [2/1000] Test Loss: 238885634048.0000 
Epoch [3/1000] Test Loss: 235835916288.0000
Epoch [4/1000] Test Loss: 237683245056.0000
Epoch [5/1000] Test Loss: 240484368384.0000
Epoch [6/1000] Test Loss: 235287920640.0000
Epoch [7/1000] Test Loss: 232444411904.0000
Epoch [8/1000] Test Loss: 233658007552.0000
Epoch [9/1000] Test Loss: 235095965696.0000
Epoch [10/1000] Test Loss: 232347713536.0000
Epoch [11/1000] Test Loss: 239848275968.0000
Epoch [12/1000] Test Loss: 242579668992.0000
Epoch [13/1000] Test Loss: 240169140224.0000
Epoch [14/1000] Test Loss: 239437922304.0000
